In [1]:
# importing the necessary libraries
import cv2
import face_recognition
import pandas as pd
import os
from PIL import Image
import time

In [2]:
# Function to load the image folder and the excel file
def load_data_from_folder(folder, metadata_file):
    images = []
    labels = []
    metadata = []

    metadata_df = pd.read_excel(metadata_file)
    metadata_dict = metadata_df.set_index("Unique ID").T.to_dict("list")
    for filename in os.listdir(folder):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            img = face_recognition.load_image_file(os.path.join(folder,filename))
            encodings = face_recognition.face_encodings(img)
            if len(encodings) > 0:
                encoding = encodings[0]
                images.append(encoding)
                labels.append(filename.split(".")[0])
                identifier = filename.split(".")[0]
                if identifier in metadata_dict:
                    metadata.append(metadata_dict[identifier])
                else:
                     metadata.append(["Unknown", "Unknown", "Unknown"])
            else:
                print(f"No face found in {filename}, skipping...")
    return images, labels, metadata

In [3]:
# Specifying the filepath
training_data_folder = "C:/Users/OWNER/Downloads/Alabi project Folder/Computer Vision/Udemy/Self Imple/Face recognition/Photos"
metadata_file = "C:/Users/OWNER/Downloads/Alabi project Folder/Computer Vision/Udemy/Self Imple/Face recognition/Face_Recognition_Training_Data.xlsx"


In [4]:
# Loading the image folder and the excel file
known_face_encodings, known_face_names, metadata = load_data_from_folder(training_data_folder, metadata_file)

In [ ]:
# Creating Access Management and Identity Verification System
video_capture = cv2.VideoCapture(0)
while True:
    ret, frame = video_capture.read()
    face_locations = face_recognition.face_locations(frame)
    face_encodings = face_recognition.face_encodings(frame, face_locations)

    for face_encoding in face_encodings:
        matches = face_recognition.compare_faces(known_face_encodings,face_encoding)
        name = "Unknown"
        full_name = "Unknown"
        age = "Unknown"
        gender = "Unknown"
        if True in matches:
            first_match_index = matches.index(True)
            name = known_face_names[first_match_index]
            full_name, age, gender = metadata[first_match_index]
            cv2.putText(frame, "Identity verified, access granted", (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,2555,0), 2, cv2.LINE_AA)
        else:
            cv2.putText(frame, "Identity not verified, access denied", (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,2555,0), 2, cv2.LINE_AA)
        # Draw rectangle around the face
        top, right, bottom, left = face_recognition.face_locations(frame)[0]
        cv2.rectangle(frame, (left, top), (right, bottom), (0,0,255),2)
        cv2.putText(frame, f"Name: {full_name}", (left, bottom + 20), cv2.FONT_HERSHEY_DUPLEX, 0.8, (255,255,255),1)
        cv2.putText(frame, f"Age: {age}", (left, bottom + 40), cv2.FONT_HERSHEY_DUPLEX, 0.8, (255,255,255),1)
        cv2.putText(frame, f"Gender: {gender}", (left, bottom + 60), cv2.FONT_HERSHEY_DUPLEX, 0.8, (255,255,255),1)
   
cv2.imshow("Video", frame)
if cv2.waitKey(1) & 0xFF == ord('q'):
    break

In [ ]:
video_capture.release()
cv2.destroyAllWindows()